In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
from src.llms import get_llms, init_langchain, LLM
from src.tuning import TuningSet
from src.experiment import (
    get_fallacy_df, save_fallacy_df, run_experiment,
    get_identification_zero_shot_prompt_template,
    get_identification_zero_shot_prompt_template_v2,
    get_identification_few_shot_prompt_template,
    get_identification_cot_prompt_template,
    get_classification_prompt_template
)

# Load environment variables from .env file
load_dotenv()

init_langchain()

# Fallacy Experiments

## Fallacy Identification

### Experiment 1.1: Fallacy Identification with zero-shot Prompt

In [2]:
filename_e11 = 'data/fallacies_e11.csv'
df_fallacies_e11 = get_fallacy_df(filename_e11)
df_fallacies_e11.head()

[2024-11-11 13:44:01] Created new fallacy identification dataframe.


,step,entity,fallacy,label,category,subcategory
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency


In [3]:
prompt_template_e11 = get_identification_zero_shot_prompt_template()
print(prompt_template_e11)

Is the following reasoning step correct? You can only answer "Yes" or "No".
[step]


In [ ]:
llms = get_llms([LLM.GPT_4O_MINI, LLM.GEMINI_1_5_FLASH, LLM.CLAUDE_3_HAIKU])

run_experiment(df_fallacies_e11, filename_e11, prompt_template_e11, llms, sleep_seconds=0.0)

save_fallacy_df(df_fallacies_e11, filename_e11)

OpenAI o1-preview cost:
- 60 zero-shot prompt responses was \$1.64
- Estimated cost for 4640 prompts is \$126.83

OpenAI o1-mini cost:
- 100 zero-shot prompt responses was \$0.38
- Estimated cost for 4640 prompts is \$17.63
- Actual cost for 4640 prompts was \$15.89

### Experiment 1.2: Fallacy Identification with few-shot Prompt

In [18]:
filename_12 = 'data/fallacies_e12.csv'
df_fallacies_e12 = get_fallacy_df(filename_12)
df_fallacies_e12.head()

[2024-10-31 14:14:12] Loaded existing fallacy dataframe from data/fallacies_e12.csv.


,step,entity,fallacy,label,category,subcategory,gpt_4o_response,claude_3_5_sonnet_response,gemini_1_5_pro_response,gpt_4o_mini_response,claude_3_haiku_response,gemini_1_5_flash_response,gemini_1_5_flash_8b_response,llama_3_1_70b_response,llama_3_1_8b_response,mistral_large_2_response,mistral_small_2_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency,No.,No.,No.,No.,No.,No,No,No.,No.,No.,No.
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency,No.,No.,No.,No.,No.,No,No,No.,No.,No.,No.
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency,No.,No.,No.,No.,No.,No,No,No.,No.,No.,No.
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency,No.,No.,Yes.,No.,No.,No,No,Yes,No.,Yes. No. No. Yes. No.,No.
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency,No.,No.,No.,No.,No.,No,No,No.,No.,No.,No.


In [19]:
prompt_template_e12 = get_identification_few_shot_prompt_template()
print(prompt_template_e12)

Is the following reasoning step correct? You can only answer "Yes" or "No".
Since if it's raining then the streets are wet and it's raining now, therefore, the streets are wet.
Yes.
Since I found a shell on the beach and this shell was beautifully shaped and colored, therefore, all shells are beautifully shaped and colored.
No.
Since I am at home or I am in the city and I am at home, therefore, I am not in the city.
No.
Since heavy snowfall often leads to traffic jams and traffic jams cause delays, therefore, heavy snowfall can lead to delays.
Yes.
[step]


In [20]:
llms = get_llms([LLM.LLAMA_3_1_70B, LLM.LLAMA_3_1_8B])

run_experiment(df_fallacies_e12, filename_12, prompt_template_e12, llms, sleep_seconds=0.2)

save_fallacy_df(df_fallacies_e12, filename_12)


### Experiment 1.3: Fallacy Identification with chain-of-thought Prompt

In [2]:
filename_e13 = 'data/fallacies_e13.csv'
df_fallacies_e13 = get_fallacy_df(filename_e13)
df_fallacies_e13.head()

[2024-11-01 08:22:52] Loaded existing fallacy dataframe from data/fallacies_e13.csv.


,step,entity,fallacy,label,category,subcategory,gpt_4o_response,claude_3_5_sonnet_response,gemini_1_5_pro_response,gpt_4o_mini_response,claude_3_haiku_response,gemini_1_5_flash_response,gemini_1_5_flash_8b_response,llama_3_1_70b_response,llama_3_1_8b_response,mistral_large_2_response,mistral_small_2_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency,No. The reasoning step is not necessarily corr...,Let's think through this step-by-step: 1. Joh...,No. Silence is not necessarily an admission o...,No.,Let's analyze this step-by-step: 1. John aske...,No. Silence doesn't necessarily mean guilt or...,No. Silence doesn't necessarily mean agreemen...,To evaluate the correctness of the reasoning s...,No. This reasoning step is incorrect because ...,"No, the reasoning step is not necessarily corr...","No, the reasoning step is not correct. Maria's..."
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency,No. The reasoning step is not correct. Bob's s...,Let's think through this step by step: 1. Ali...,No. Silence could indicate a lack of knowledg...,No.,Let's analyze this step-by-step: 1. Alice ask...,No. Silence doesn't imply knowledge. Bob's s...,No. Silence does not equate to knowledge. Bo...,"No, the reasoning step is not correct. The c...",No. This reasoning is incorrect because remai...,No. Bob's silence does not necessarily indicat...,"No, the reasoning step is not correct. Bob's s..."
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency,No. The reasoning step is not correct. The con...,Let's think through this step-by-step: 1. Ali...,No. Bob's silence doesn't confirm Alice's cla...,No.,Let's analyze this step-by-step: 1. Alice cla...,No. Bob's silence doesn't provide evidence su...,"No. Alice's claim, even if true, and Bob's si...",To evaluate the correctness of the reasoning s...,No. This reasoning is incorrect because it as...,"No, the reasoning step is not correct. Bob's s...","No, the reasoning step is not correct. Just be..."
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency,No. The reasoning step is not necessarily corr...,Let's think through this step-by-step: 1. The...,"No. While Tom's experience is valuable, the a...",No.,"Let's analyze this step-by-step: 1. Tom, a se...",No. The reasoning commits an appeal to author...,No. Just because one person says something an...,To evaluate the correctness of the reasoning s...,No. The reasoning step is incorrect for sever...,"No, the reasoning step is not necessarily corr...","No, the reasoning step is not correct. Here's ..."
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency,No. The reasoning step is not correct. The fac...,Let's think through this step-by-step: 1. Joh...,No. Silence does not imply guilt or fear. Ma...,No.,Let's analyze this step-by-step: 1. John accu...,No. Silence does not equal guilt or fear. Ma...,No. Silence in the face of an accusation does...,"No, the reasoning step is not correct. Here'...",No. This reasoning step is incorrect because ...,No. Mary's silence does not necessarily mean s...,"No, the reasoning step is not correct. Just be..."


In [3]:
prompt_template_e13 = get_identification_cot_prompt_template()
print(prompt_template_e13)

Is the following reasoning step correct?
Let's think step by step and then answer "Yes" or "No".
[step]


In [4]:
llms = get_llms([LLM.LLAMA_3_1_70B, LLM.LLAMA_3_1_8B])

run_experiment(df_fallacies_e13, filename_e13, prompt_template_e13, llms, sleep_seconds=0.2)

save_fallacy_df(df_fallacies_e13, filename_e13)


### Experiment 1.4: Fallacy Identification with Fine-Tuning

In [2]:
filename_e14 = 'data/fallacies_e14.csv'
df_fallacies_e14 = get_fallacy_df(filename_e14)

# Select only test set
df_fallacies_e14 = df_fallacies_e14[df_fallacies_e14['tuning'] == TuningSet.TEST.value]

[2024-11-05 18:13:57] Loaded existing fallacy dataframe from data/fallacies_e14.csv.


In [2]:
prompt_template_e14 = get_identification_zero_shot_prompt_template()
print(prompt_template_e14)

NameError: name 'get_identification_zero_shot_prompt_template' is not defined

In [4]:
llms = get_llms([LLM.GPT_4O_MINI_IDENTIFICATION])

run_experiment(df_fallacies_e14, filename_e14, prompt_template_e14, llms, sleep_seconds=0.1)

save_fallacy_df(df_fallacies_e14, filename_e14)

[2024-11-05 18:15:13] Processed 100 responses for LLM gpt_4o_mini_identification (index=199).
[2024-11-05 18:16:24] Processed 200 responses for LLM gpt_4o_mini_identification (index=399).
[2024-11-05 18:17:34] Processed 300 responses for LLM gpt_4o_mini_identification (index=599).
[2024-11-05 18:18:50] Processed 400 responses for LLM gpt_4o_mini_identification (index=799).
[2024-11-05 18:19:58] Processed 500 responses for LLM gpt_4o_mini_identification (index=999).
[2024-11-05 18:21:05] Processed 600 responses for LLM gpt_4o_mini_identification (index=1199).
[2024-11-05 18:22:15] Processed 700 responses for LLM gpt_4o_mini_identification (index=1399).
[2024-11-05 18:23:23] Processed 800 responses for LLM gpt_4o_mini_identification (index=1599).
[2024-11-05 18:24:28] Processed 900 responses for LLM gpt_4o_mini_identification (index=1799).
[2024-11-05 18:25:35] Processed 1000 responses for LLM gpt_4o_mini_identification (index=1999).
[2024-11-05 18:26:43] Processed 1100 responses for LLM

### Experiment 1.5: Fallacy Identification with rephrased zero-shot Prompt


In [8]:
filename_e15 = 'data/fallacies_e15.csv'
df_fallacies_e15 = get_fallacy_df(filename_e15)
df_fallacies_e15.head()

[2024-11-11 12:03:39] Loaded existing fallacy dataframe from data/fallacies_e15.csv.


,step,entity,fallacy,label,category,subcategory,claude_3_5_sonnet_response,gpt_4o_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency,Yes,Yes
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency,Yes,Yes
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency,Yes,Yes
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency,Yes,Yes
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency,Yes,Yes


In [9]:
prompt_template_e15 = get_identification_zero_shot_prompt_template_v2()
print(prompt_template_e15)

Does the following reasoning step contain a logical fallacy? You can only answer "Yes" or "No".
[step]


In [10]:
llms = get_llms([LLM.GPT_4O, LLM.GEMINI_1_5_FLASH])

run_experiment(df_fallacies_e15, filename_e15, prompt_template_e15, llms, sleep_seconds=0.1)

save_fallacy_df(df_fallacies_e15, filename_e15)

[2024-11-11 12:04:42] Processed 100 responses for LLM gpt_4o (index=409).
[2024-11-11 12:05:42] Processed 200 responses for LLM gpt_4o (index=509).
[2024-11-11 12:06:39] Processed 300 responses for LLM gpt_4o (index=609).
[2024-11-11 12:07:43] Processed 400 responses for LLM gpt_4o (index=709).
[2024-11-11 12:08:45] Processed 500 responses for LLM gpt_4o (index=809).
[2024-11-11 12:09:55] Processed 600 responses for LLM gpt_4o (index=909).
[2024-11-11 12:10:55] Processed 700 responses for LLM gpt_4o (index=1009).
[2024-11-11 12:11:58] Processed 800 responses for LLM gpt_4o (index=1109).
[2024-11-11 12:13:06] Processed 900 responses for LLM gpt_4o (index=1209).
[2024-11-11 12:14:04] Processed 1000 responses for LLM gpt_4o (index=1309).
[2024-11-11 12:15:08] Processed 1100 responses for LLM gpt_4o (index=1409).
[2024-11-11 12:16:07] Processed 1200 responses for LLM gpt_4o (index=1509).
[2024-11-11 12:17:05] Processed 1300 responses for LLM gpt_4o (index=1609).
[2024-11-11 12:18:14] Proce

### Experiment 1.6: Fallacy Identification with zero-shot Prompt (Replication)

Are the results from the previous experiment replicable? Since temperature is set to 0, the models should produce the same output for the same input.

In [5]:
filename_e16 = 'data/fallacies_e16.csv'
df_fallacies_e16 = get_fallacy_df(filename_e16)
df_fallacies_e16.head()

[2024-11-11 13:48:48] Loaded existing fallacy dataframe from data/fallacies_e16.csv.


,step,entity,fallacy,label,category,subcategory,gpt_4o_mini_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency,No.
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency,No.
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency,No.
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency,No.
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency,No.


In [6]:
prompt_template_e16 = get_identification_zero_shot_prompt_template()
print(prompt_template_e16)

Is the following reasoning step correct? You can only answer "Yes" or "No".
[step]


In [7]:
llms = get_llms([LLM.GPT_4O_MINI, LLM.GEMINI_1_5_FLASH, LLM.CLAUDE_3_HAIKU])

run_experiment(df_fallacies_e16, filename_e16, prompt_template_e16, llms, sleep_seconds=0.0)

save_fallacy_df(df_fallacies_e16, filename_e16)

[2024-11-11 13:50:07] Processed 100 responses for LLM gpt_4o_mini (index=289).
[2024-11-11 13:51:01] Processed 200 responses for LLM gpt_4o_mini (index=389).
[2024-11-11 13:51:50] Processed 300 responses for LLM gpt_4o_mini (index=489).
[2024-11-11 13:52:37] Processed 400 responses for LLM gpt_4o_mini (index=589).
[2024-11-11 13:53:21] Processed 500 responses for LLM gpt_4o_mini (index=689).
[2024-11-11 13:54:12] Processed 600 responses for LLM gpt_4o_mini (index=789).
[2024-11-11 13:55:00] Processed 700 responses for LLM gpt_4o_mini (index=889).
[2024-11-11 13:55:51] Processed 800 responses for LLM gpt_4o_mini (index=989).
[2024-11-11 13:56:38] Processed 900 responses for LLM gpt_4o_mini (index=1089).
[2024-11-11 13:57:24] Processed 1000 responses for LLM gpt_4o_mini (index=1189).
[2024-11-11 13:58:13] Processed 1100 responses for LLM gpt_4o_mini (index=1289).
[2024-11-11 13:59:01] Processed 1200 responses for LLM gpt_4o_mini (index=1389).
[2024-11-11 13:59:52] Processed 1300 response

## Fallacy Classification

### Experiment 2.1: Fallacy Classification with zero-shot Prompt

In [9]:
filename_21 = 'data/fallacies_e21.csv'
df_fallacies_e21 = get_fallacy_df(filename_21, only_incorrect=True)

df_fallacies_e21.head()

[2024-11-01 08:38:53] Loaded existing fallacy dataframe from data/fallacies_e21.csv.


,step,entity,fallacy,label,category,subcategory,gpt_4o_response,gpt_4_response,gpt_4o_mini_response,claude_3_5_sonnet_response,claude_3_haiku_response,gemini_1_5_pro_response,gemini_1_5_flash_8b_response,o1_preview_response,mistral_large_2_response,mistral_small_2_response,llama_3_1_70b_response
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency,Argument from Silence,Argument from Silence,Affirming the Consequent,Argument from Silence,Denying the Antecedent,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency,Argument from Silence,Argument from Silence,Affirming the Consequent,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,(164) Argument from Silence,Argument from Silence
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Appeal to Silence,Argument from Silence
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency,Argument from Silence,(79) Appeal to Authority,Appeal to Authority,Appeal to Authority,Appeal to Authority,Appeal to Authority,Appeal to Authority,Argument from Silence,Argument from Silence,Appeal to Authority,Argument from Silence
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence,Argument from Silence


In [10]:
prompt_template_e21 = get_classification_prompt_template()
print(prompt_template_e21)

You are a logical fallacy classifier. Given an incorrect reasoning step, your task is to identify its type of fallacy.
Answer by choosing one of these fallacies:
(1) Affirming the Consequent
(2) Denying the Antecedent
(3) Negating Antecedent and Consequent
(4) Commutation of Conditionals
(5) Affirming a Disjunct
(6) Denying a Conjunct
(7) Fallacy of the Undistributed Middle
(8) Exclusive Premises
(9) Fallacy of Four Terms
(10) Illicit Substitution of Identicals
(11) Illicit Minor
(12) Illicit Major
(13) Negative Conclusion from Affirmative Premises
(14) Affirmative Conclusion from a Negative Premise
(15) False Conversion
(16) Unwarranted Contrast
(17) Quantifier Shift Fallacy
(18) Existential Fallacy
(19) Fallacy of Every and All
(20) Illicit Contraposition
(21) Gamblers Fallacy
(22) Hot Hand Fallacy
(23) Conjunction Fallacy
(24) Disjunction Fallacy
(25) Argument of the Beard
(26) Appeal to Extremes
(27) Type Token Fallacy
(28) Use Mention Error
(29) Reification
(30) Fake Precision
(31

In [11]:
llms = get_llms([LLM.LLAMA_3_1_70B, LLM.LLAMA_3_1_8B])

run_experiment(df_fallacies_e21, filename_21, prompt_template_e21, llms, sleep_seconds=0.1)

save_fallacy_df(df_fallacies_e21, filename_21)

[2024-11-01 08:41:10] Processed 100 responses for LLM llama_3_1_70b (index=338).
[2024-11-01 08:43:39] Processed 200 responses for LLM llama_3_1_70b (index=438).
[2024-11-01 08:46:31] Processed 300 responses for LLM llama_3_1_70b (index=538).
[2024-11-01 08:48:54] Processed 400 responses for LLM llama_3_1_70b (index=638).
[2024-11-01 08:51:39] Processed 500 responses for LLM llama_3_1_70b (index=738).
[2024-11-01 08:54:39] Processed 600 responses for LLM llama_3_1_70b (index=838).
[2024-11-01 08:57:17] Processed 700 responses for LLM llama_3_1_70b (index=938).
[2024-11-01 08:59:49] Processed 800 responses for LLM llama_3_1_70b (index=1038).
[2024-11-01 09:02:41] Processed 900 responses for LLM llama_3_1_70b (index=1138).
[2024-11-01 09:05:25] Processed 1000 responses for LLM llama_3_1_70b (index=1238).
[2024-11-01 09:08:19] Processed 1100 responses for LLM llama_3_1_70b (index=1338).
[2024-11-01 09:11:05] Processed 1200 responses for LLM llama_3_1_70b (index=1438).
[2024-11-01 09:13:50

### Experiment 2.2: Fallacy Classification with Fine-Tuning

In [3]:
filename_e22 = 'data/fallacies_e22.csv'
df_fallacies_e22 = get_fallacy_df(filename_e22, only_incorrect=True)

# Select only test set
df_fallacies_e22 = df_fallacies_e22[df_fallacies_e22['tuning'] == TuningSet.TEST.value]

[2024-10-28 14:41:15] Loaded existing fallacy dataframe from data/fallacies_e22.csv.


In [ ]:
prompt_template_e22 = get_classification_prompt_template()
print(prompt_template_e22)

In [4]:

llms = get_llms([LLM.GPT_4O_MINI_CLASSIFICATION])

run_experiment(df_fallacies_e22, filename_e22, prompt_template_e22, llms, sleep_seconds=0.5)

save_fallacy_df(df_fallacies_e22, filename_e22)

[2024-10-28 14:52:57] Processed 100 responses for LLM gpt_4o_mini_tuned_v1 (index=199).
[2024-10-28 14:54:09] Processed 200 responses for LLM gpt_4o_mini_tuned_v1 (index=399).
[2024-10-28 14:55:21] Processed 300 responses for LLM gpt_4o_mini_tuned_v1 (index=599).
[2024-10-28 14:56:34] Processed 400 responses for LLM gpt_4o_mini_tuned_v1 (index=799).
[2024-10-28 14:57:48] Processed 500 responses for LLM gpt_4o_mini_tuned_v1 (index=999).
[2024-10-28 14:58:59] Processed 600 responses for LLM gpt_4o_mini_tuned_v1 (index=1199).
[2024-10-28 15:00:17] Processed 700 responses for LLM gpt_4o_mini_tuned_v1 (index=1399).
[2024-10-28 15:01:31] Processed 800 responses for LLM gpt_4o_mini_tuned_v1 (index=1599).
[2024-10-28 15:02:52] Processed 900 responses for LLM gpt_4o_mini_tuned_v1 (index=1799).
[2024-10-28 15:04:13] Processed 1000 responses for LLM gpt_4o_mini_tuned_v1 (index=1999).
[2024-10-28 15:05:30] Processed 1100 responses for LLM gpt_4o_mini_tuned_v1 (index=2199).
